In [ ]:
# Process Mining
## Trace Clustering 

In [ ]:
import pm4py
import pandas as pd

In [ ]:
event_log = pm4py.read_xes('logs/sintetico_completo.xes', sep=',')
#lawsuit_id,date,movement,order,status,area,subject_matter,control,digital,class,distribution_date,court_department,claim_amount,judge
event_log = pm4py.format_dataframe(event_log, case_id='case:concept:name', activity_key='Activity', timestamp_key='time:timestamp')
event_log['time:timestamp']=pd.to_datetime(event_log['time:timestamp'], utc=True) # format the timestamp column

In [ ]:
event_log.head()

In [ ]:
_sint1 = event_log[event_log['case:concept:name'] =='2778'] 
_sint1.to_csv('_sint1.csv')
_sint1

In [ ]:
_sint2 = event_log[event_log['case:concept:name'] =='1956'] 
_sint2.to_csv('_sint2.csv')
_sint2

In [ ]:
_6686 = event_log[event_log['case:concept:name'] =='6686'] 
_6686.to_csv('_6686.csv')
_6686

In [ ]:
### Exploiting Pandas

In [ ]:
#### Number of Cases

In [ ]:
event_log['elementId'].nunique()

In [ ]:
#### Activity Ocurrences

In [ ]:
event_log["concept:name"].value_counts()

In [ ]:
#### Activities in Cases

In [ ]:
event_log.groupby(["elementId", "concept:name"]).first().reset_index()["concept:name"].value_counts()

In [ ]:
#### Start Activities

In [ ]:
event_log.groupby("elementId").first()["concept:name"].value_counts()

In [ ]:
#### End Activities

In [ ]:
event_log.groupby("elementId").last()["concept:name"].value_counts()

In [ ]:
## Data Exploration

In [ ]:
pm4py.view_events_distribution_graph(event_log, distr_type="days_week") # Matplotlib inside!

In [ ]:
#### Events per Hour

In [ ]:
pm4py.view_events_distribution_graph(event_log, distr_type="hours") # Matplotlib inside!

In [ ]:
#### Event Distribution Plot

In [ ]:
pm4py.view_events_per_time_graph(event_log) # Matplotlib inside!

In [ ]:
#### Case Duration Distribution

In [ ]:
pm4py.view_case_duration_graph(event_log) # Matplotlib inside!

In [ ]:
## Fun with Process Maps (and Graphy Theory)

In [ ]:
dfg = pm4py.discover_dfg(event_log)
print(dfg[0]) # graph structure

In [ ]:
#### Start Activities

In [ ]:
print(dfg[1]) # start activities

In [ ]:
#### End Activities

In [ ]:
print(dfg[2]) # end activities

In [ ]:
#### Visualizing the Process Map

In [ ]:
pm4py.view_dfg(dfg[0], dfg[1], dfg[2])

In [ ]:
### Graph Theory for Process Maps (using NetworkX)


In [ ]:
#### Converting the Process Map

In [ ]:
%matplotlib inline
import networkx as nx
import itertools as it
G = nx.DiGraph()
alphabet = set(list(it.chain.from_iterable([[a,b] for (a,b) in dfg[0]]))).union(dfg[1].keys()).union(dfg[2].keys())
G.add_nodes_from(alphabet) # adding nodes
nx.draw(G, with_labels = True)

In [ ]:
#### Adding Arcs

In [ ]:
G.add_edges_from(dfg[0].keys())
nx.draw(G, with_labels = True)

In [ ]:
### Degree of Vertices

In [ ]:
sorted([(n, i) for (n,i) in G.in_degree()], key=lambda t: t[1], reverse=True)

In [ ]:
### Dominator of a Graph

In [ ]:
nx.immediate_dominators(G, 'Decisão')

In [ ]:
### Centrality of a Node

In [ ]:
sorted(nx.degree_centrality(G).items(), key=lambda t: t[1], reverse=True) 

In [ ]:
## Discovering Process Models

In [ ]:
### Discovering a BPMN Model (Raw Data)

In [ ]:
### Discovering a BPMN Model (Internal Filtering)

In [ ]:
bpmn_2 = pm4py.discover_bpmn_inductive(event_log, noise_threshold=0.1)
pm4py.view_bpmn(bpmn_2)

In [ ]:
from pm4py.objects.bpmn.exporter import exporter
exporter.apply(bpmn_2, "juris.bpmn")

In [ ]:
## Checking Model Quality

In [ ]:
### Converting the Model

In [ ]:
pn, im, fm = pm4py.convert_to_petri_net(bpmn_2)
pm4py.view_petri_net(pn, im, fm)

In [ ]:
### Checking Behavioral Model Properties

In [ ]:
pm4py.check_soundness(pn, im, fm)

In [ ]:
### Computing Quality Statistics

In [ ]:
#### Fitness

In [ ]:
pm4py.fitness_token_based_replay(event_log, pn, im, fm) # fast results; less accurate/explainable

In [ ]:
pm4py.fitness_alignments(event_log, pn, im, fm)

In [ ]:
#### Precision

In [ ]:
pm4py.precision_alignments(event_log, pn, im, fm)

In [ ]:
#### Diagnostics

In [ ]:
alignments = pm4py.conformance_diagnostics_alignments(event_log, pn, im, fm)
alignments[0]

In [ ]:
non_fitting = list(filter(lambda a: a['fitness']<1.0, alignments))
non_fitting[0]

In [ ]:
#### Reconstructing the Model Behavior

In [ ]:
trace = list(map(lambda m: m[1], filter(lambda m: m[1] !='>>' and m[1] is not None, alignment)))
trace

In [ ]:
### Comparing Footprints

In [ ]:
from pm4py.algo.discovery.footprints import algorithm as footprints_discovery
process_tree = pm4py.discover_process_tree_inductive(event_log, noise_threshold=0.1)
fp_log = footprints_discovery.apply(event_log, variant=footprints_discovery.Variants.ENTIRE_EVENT_LOG)
fp_model = footprints_discovery.apply(process_tree, variant=footprints_discovery.Variants.PROCESS_TREE)
from pm4py.visualization.footprints import visualizer as fp_visualizer
gviz = fp_visualizer.apply(fp_log, fp_model)
fp_visualizer.view(gviz)

In [ ]:
### Machine Learning at the Case Level

In [ ]:
#### Attribute Inspection

In [ ]:
pm4py.get_event_attributes(event_log)

In [ ]:
pm4py.get_trace_attributes(event_log)

In [ ]:
#### Creating a Feature Table

In [ ]:
from pm4py.algo.transformation.log_to_features import algorithm as log_to_features
ft, feature_names = log_to_features.apply(event_log, parameters={"str_ev_attr": ["lawsuit_id"],
                                                "str_tr_attr": ["concept:name"],
                                                "num_ev_attr": [], "num_tr_attr": [],
                                                "str_evsucc_attr": ["lawsuit_id"]})
pd.DataFrame(ft, columns=feature_names)

In [ ]:
#### Principal Component Analysis

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=5)
pca_df = pd.DataFrame(pca.fit_transform(ft))
pca_df.rename({c: str(c) for c in pca_df.columns}, axis='columns') # renaming the columns to string
pca_df

In [ ]:
#### Outlier Detection

In [ ]:
from sklearn.ensemble import IsolationForest
model=IsolationForest()
model.fit(pca_df)
pca_df["scores"] = model.decision_function(pca_df)
pca_df.sort_values("scores")

In [ ]:
#### Concept Drift Detection

In [ ]:
from pm4py.algo.transformation.log_to_features.util import locally_linear_embedding
from pm4py.visualization.graphs import visualizer

x, y = locally_linear_embedding.apply(event_log)
gviz = visualizer.apply(x, y, variant=visualizer.Variants.DATES, parameters={"title": "Locally Linear Embedding", "y_axis": "Intensity"})
visualizer.view(gviz)

In [ ]:
#### Clustering

In [ ]:
from sklearn.cluster import KMeans
clustering = KMeans(n_clusters=4)
clustering.fit_predict(ft)

In [ ]:
labels = clustering.labels_
centroids = clustering.cluster_centers_

l = clustering.fit_predict(ft)
pca = PCA(n_components=2).fit(ft)
datapoint = pca.transform(ft)
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure
label1 = ["#FFFF00", "#008000", "#0000FF", "#800080", "#800d00"]
color = [label1[i] for i in labels]
plt.scatter(datapoint[:, 0], datapoint[:, 1], c=color)
centroidpoint = pca.transform(centroids)
plt.scatter(centroidpoint[:, 0], centroidpoint[:, 1], marker='^', s=150, c='#000000')
plt.show()

In [ ]:
#### Decision Point Mining

In [ ]:
from pm4py.algo.decision_mining import algorithm as dem
ft = dem.apply(event_log, pn, im, fm, decision_point='exi_id6046d342-19d8-46a9-a7f1-af0f14e43653')
ft[0]

In [ ]:
ft[1]

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf.fit(ft[0], ft[1])
tree.plot_tree(clf, 
                   feature_names=list(ft[0].columns.values),  
                   class_names=['T13 Adjust document X request unlicensed', 'T11 Create document X request unlicensed'],
                   filled=True)

In [ ]:
#### Case Duration Prediction

In [ ]:
data, feature_names = log_to_features.apply(event_log, parameters={"str_ev_attr": ["lawsuit_id"],
                                                "str_tr_attr": ["concept:name"],
                                                "num_ev_attr": [], "num_tr_attr": [],
                                                "str_evsucc_attr": ["lawsuit_id"]})
#throughput_time = [y[-1]["date"].timestamp() - y[0]["date"].timestamp() for y in event_log]
#throughput_time[0:5]
data

In [ ]:
import random
training_cases = set(random.sample([i for i in range(len(event_log))], 500))
data_training = [data[i] for i in range(len(event_log)) if i in training_cases]
throughput_time_training = [throughput_time[i] for i in range(len(event_log)) if i in training_cases]

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
regressor = KNeighborsRegressor(n_neighbors=3)
regressor.fit(data_training, throughput_time_training)

In [ ]:
data_validation = [data[i] for i in range(len(event_log)) if i not in training_cases]
throughput_time_validation = [throughput_time[i] for i in range(len(event_log)) if i not in training_cases]

In [ ]:
predicted_throughput_time = regressor.predict(data_validation)

In [ ]:
for i in range(5):
    print("case actual throughput=", throughput_time_validation[i], " predicted throughput = ", predicted_throughput_time[i])